In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [2]:
data = pd.read_csv('MF_data.csv')
data

Company        Time  Total Assets  Total Liabilities  Total Equity  \
0        DVA  2019-12-01       17311.4            13811.8        3499.6   
1        CVS  2012-12-01       66221.0            28568.0       37653.0   
2        NEM  2014-12-01       24916.0            11827.0       13089.0   
3       TTWO  2019-03-01        4243.1             2202.5        2040.6   
4        ADS  2018-12-01       30387.7            28055.6        2332.1   
5         MU  2011-09-01       14752.0             4900.0        9852.0   
6        ALK  2015-12-01        6530.0             4119.0        2411.0   
7        KMX  2019-02-01       18717.9            15360.8        3357.0   
8      MRWSY  2011-01-01       14493.5             5907.3        8586.1   
9        ETR  2018-12-01       48275.1            39211.4        9063.7   
10      TTWO  2012-03-01        1149.4              553.7         595.7   
11        WU  2010-12-01        7929.2             7346.5         582.7   
12       TDS  2017-12-01        9295.0             4402.0        4893.0   
13       CMC  2011-08-01        3683.1             2522.5        1160.6   
14       AOS  2017-12-01        3197.4             1552.5        1644.9   
15       XOM  2013-12-01      346808.0           166313.0      180495.0   
16       IFF  2017-12-01        4598.9             2909.6        1689.3   
17     MRWSY  2019-02-01       13951.8             8294.4        5657.4   
18       LUV  2017-12-01       25110.0            15469.0        9641.0   
19       CTL  2018-12-01       70256.0            50428.0       19828.0   
20      FAST  2010-12-01        1468.3              185.8        1282.5   
21        WU  2017-12-01        9231.4             9722.8        -491.4   
22       GPC  2011-12-01        6202.8             3449.2        2753.6   
23        WM  2019-12-01       27743.0            20673.0        7070.0   
24      GPRE  2013-12-01        1532.0              986.7         545.4   
25       LOW  2013-02-01       32666.0            18809.0       13857.0   
26       IFF  2010-12-01        2872.5             1869.3        1003.2   
27      ULTA  2016-01-01        2230.9              788.0        1442.9   
28       SWK  2015-01-01       15849.1             9337.2        6511.9   
29       APD  2010-09-01       13505.9             7808.3        5697.6   
...      ...         ...           ...                ...           ...   
4970     BLK  2012-12-01      200451.0           174834.0       25617.0   
4971     TFX  2017-12-01        6181.5             3751.0        2430.5   
4972   CPYYY  2019-12-01       24048.5            21670.7        2377.8   
4973    ALGN  2011-12-01         649.3              158.5         490.8   
4974    SOHU  2012-12-01        2082.6              705.6        1377.0   
4975    FAST  2017-12-01        2910.5              813.6        2096.9   
4976     ETR  2015-12-01       44647.7            35072.7        9575.0   
4977      HD  2019-02-01       44003.0            45881.0       -1878.0   
4978     CNK  2013-12-01        4144.2             3041.7        1102.4   
4979     CVX  2016-12-01      260078.0           113356.0      146722.0   
4980     ALB  2010-12-01        3068.1             1592.3        1475.7   
4981    TSCO  2014-12-01        2034.6              741.0        1293.6   
4982     PEG  2018-12-01       45326.0            30949.0       14377.0   
4983     GTN  2017-12-01        3260.9             2268.0         992.9   
4984     AAP  2017-12-01        8482.3             5067.1        3415.2   
4985      NI  2010-12-01       19913.4            15015.9        4897.5   
4986     SHW  2015-12-01        5778.9             4911.0         867.9   
4987     DOV  2013-12-01       10855.2             5477.8        5377.4   
4988     ALK  2016-12-01        9962.0             7031.0        2931.0   
4989      LH  2015-12-01       14104.7             9144.7        4960.0   
4990     ITW  2019-12-01       15068.0            12038.0        3030.0   
4991     JNJ  2017-12-01      

In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data


Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
Total Revenues
Done
Operating Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done
Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


Total Assets  Total Liabilities  Total Equity  \
0         0.019172           0.016582      0.034032   
1         0.073376           0.034300      0.111647   
2         0.027600           0.014198      0.055825   
3         0.004689           0.002642      0.030717   
4         0.033664           0.033685      0.031379   
5         0.016336           0.005881      0.048468   
6         0.007224           0.004943      0.031559   
7         0.020731           0.018442      0.033708   
8         0.016049           0.007091      0.045592   
9         0.053488           0.047080      0.046677   
10        0.001261           0.000662      0.027433   
11        0.008774           0.008819      0.027404   
12        0.010288           0.005283      0.037199   
13        0.004069           0.003026      0.028717   
14        0.003530           0.001862      0.029818   
15        0.384336           0.199693      0.436259   
16        0.005084           0.003491      0.029918   
17        0.015449           0.009957      0.038936   
18        0.027815           0.018571      0.047989   
19        0.077848           0.060548      0.071139   
20        0.001614           0.000221      0.028994   
21        0.010217           0.011672      0.024963   
22        0.006861           0.004139      0.032337   
23        0.030733           0.024820      0.042146   
24        0.001685           0.001182      0.027319   
25        0.036189           0.022582      0.057570   
26        0.003170           0.002242      0.028359   
27        0.002459           0.000944      0.029359   
28        0.017552           0.011209      0.040878   
29        0.014955           0.009373      0.039027   
...            ...                ...           ...   
4970      0.222136           0.209925      0.084295   
4971      0.006837           0.004501      0.031603   
4972      0.026639           0.026018      0.031483   
4973      0.000706           0.000188      0.027195   
4974      0.002295           0.000845      0.029209   
4975      0.003212           0.000974      0.030845   
4976      0.049468           0.042110      0.047839   
4977      0.048753           0.055088      0.021812   
4978      0.004580           0.003650      0.028585   
4979      0.288218           0.136107      0.359509   
4980      0.003387           0.001909      0.029433   
4981      0.002242           0.000887      0.029019   
4982      0.050219           0.037159      0.058752   
4983      0.003601           0.002721      0.028336   
4984      0.009387           0.006082      0.033841   
4985      0.022056           0.018027      0.037209   
4986      0.006391           0.005894      0.028052   
4987      0.012017           0.006575      0.038300   
4988      0.011027           0.008440      0.032740   
4989      0.015618           0.010978      0.037351   
4990      0.016686           0.014452      0.032965   
4991      0.174318           0.116639      0.162795   
4992      0.006609           0.004069      0.031954   
4993      0.016173           0.012405      0.035788   
4994      0.492156           0.362724      0.348794   
4995      0.065951           0.059137      0.049413   
4996      0.003041           0.002425      0.027748   
4997      0.001258           0.000380      0.027962   
4998      0.004523           0.002973      0.029751   
4999      0.001745           0.001207      0.027396   

      Total Shares Out. on Filing Date  Book Value / Share  \
0                             0.001569            0.000400   
1                             0.015476            0.000453   
2                             0.006265            0.000414   
3                             0.001328            0.000409   
4                             0.000655            0.000503   
5                             0.012412            0.000369   
6                             0.001557            0.000409   
7                             0.002079            0.000412   
8                             0.03325

In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:26] # Parameters
y = NpMatrix[:,26] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[1.91721365e-02 1.65816374e-02 3.40324380e-02 ... 4.83453945e-01
  4.77627905e-01 4.27248134e-03]
 [7.33761036e-02 3.42997600e-02 1.11646922e-01 ... 4.80980464e-01
  4.75545551e-01 3.90452105e-03]
 [2.75999194e-02 1.41984405e-02 5.58245974e-02 ... 4.62101128e-01
  4.55867310e-01 3.76725837e-03]
 ...
 [1.25797232e-03 3.80029127e-04 2.79620670e-02 ... 4.58195791e-01
  4.50900643e-01 3.74751181e-03]
 [4.52309262e-03 2.97263226e-03 2.97505448e-02 ... 4.58272724e-01
  4.51238899e-01 3.81734721e-03]
 [1.74549062e-03 1.20720785e-03 2.73962081e-02 ... 4.57048890e-01
  4.49811725e-01 3.79471091e-03]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 26
Total Number of Training instances: 5000


[75.03 48.35 18.44 ...  8.71 47.18 33.97]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 75.03
Total number of y values 5000


In [6]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=26, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [7]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [8]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 2s 487us/sample - loss: 57.5053 - mae: 57.5054 - mse: 21710.4668
Epoch 2/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 47.3182 - mae: 47.3182 - mse: 19567.6758
Epoch 3/1000
4000/4000 [==============================] - 1s 317us/sample - loss: 47.0383 - mae: 47.0383 - mse: 19480.5137
Epoch 4/1000
4000/4000 [==============================] - 1s 266us/sample - loss: 46.9378 - mae: 46.9378 - mse: 19468.7109
Epoch 5/1000
4000/4000 [==============================] - 1s 242us/sample - loss: 46.7349 - mae: 46.7349 - mse: 19488.3887
Epoch 6/1000
4000/4000 [==============================] - 1s 231us/sample - loss: 46.2519 - mae: 46.2519 - mse: 19389.0762
Epoch 7/1000
4000/4000 [==============================] - 1s 223us/sample - loss: 46.3533 - mae: 46.3533 - mse: 19452.6719
Epoch 8/1000
4000/4000 [==============================] - 1s 242us/sample - loss: 45.9821 - mae: 45.9820 - mse: 19440

4000/4000 [==============================] - 2s 412us/sample - loss: 42.0615 - mae: 42.0615 - mse: 19173.7012
Epoch 67/1000
4000/4000 [==============================] - 2s 436us/sample - loss: 42.2880 - mae: 42.2880 - mse: 19273.0938
Epoch 68/1000
4000/4000 [==============================] - 1s 360us/sample - loss: 42.1356 - mae: 42.1355 - mse: 19231.9004
Epoch 69/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 42.0666 - mae: 42.0666 - mse: 19182.6426
Epoch 70/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 41.9533 - mae: 41.9533 - mse: 19188.0352
Epoch 71/1000
4000/4000 [==============================] - 1s 342us/sample - loss: 42.3797 - mae: 42.3797 - mse: 19254.9062
Epoch 72/1000
4000/4000 [==============================] - 1s 322us/sample - loss: 42.1090 - mae: 42.1091 - mse: 19217.9199
Epoch 73/1000
4000/4000 [==============================] - 2s 413us/sample - loss: 41.8955 - mae: 41.8955 - mse: 19202.0078
Epoch 74/1000
4000/400

4000/4000 [==============================] - 1s 346us/sample - loss: 41.6059 - mae: 41.6059 - mse: 19144.4082
Epoch 132/1000
4000/4000 [==============================] - 1s 345us/sample - loss: 41.3627 - mae: 41.3627 - mse: 19098.2168
Epoch 133/1000
4000/4000 [==============================] - 1s 332us/sample - loss: 41.5621 - mae: 41.5620 - mse: 19110.8066
Epoch 134/1000
4000/4000 [==============================] - 1s 309us/sample - loss: 41.4339 - mae: 41.4339 - mse: 19090.9648
Epoch 135/1000
4000/4000 [==============================] - 1s 343us/sample - loss: 41.4368 - mae: 41.4367 - mse: 19032.2754
Epoch 136/1000
4000/4000 [==============================] - 1s 315us/sample - loss: 41.4282 - mae: 41.4282 - mse: 19135.9863
Epoch 137/1000
4000/4000 [==============================] - 1s 347us/sample - loss: 41.6123 - mae: 41.6123 - mse: 19114.9824
Epoch 138/1000
4000/4000 [==============================] - 1s 319us/sample - loss: 41.4056 - mae: 41.4056 - mse: 19083.6328
Epoch 139/1000


4000/4000 [==============================] - 2s 394us/sample - loss: 40.8522 - mae: 40.8522 - mse: 18944.2324
Epoch 197/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 40.6621 - mae: 40.6621 - mse: 18890.8984
Epoch 198/1000
4000/4000 [==============================] - 2s 459us/sample - loss: 40.7572 - mae: 40.7572 - mse: 18914.4570
Epoch 199/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 40.8079 - mae: 40.8079 - mse: 18954.3418
Epoch 200/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 40.6521 - mae: 40.6521 - mse: 18843.3438
Epoch 201/1000
4000/4000 [==============================] - 2s 444us/sample - loss: 40.7321 - mae: 40.7321 - mse: 18943.0840
Epoch 202/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 40.7856 - mae: 40.7856 - mse: 18860.4316
Epoch 203/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 40.6973 - mae: 40.6972 - mse: 18927.1719
Epoch 204/1000


4000/4000 [==============================] - 1s 367us/sample - loss: 40.2727 - mae: 40.2727 - mse: 18821.6602
Epoch 262/1000
4000/4000 [==============================] - 2s 403us/sample - loss: 40.2663 - mae: 40.2663 - mse: 18787.2949
Epoch 263/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 39.9153 - mae: 39.9153 - mse: 18694.1992
Epoch 264/1000
4000/4000 [==============================] - 1s 362us/sample - loss: 39.8156 - mae: 39.8156 - mse: 18662.4277
Epoch 265/1000
4000/4000 [==============================] - 1s 372us/sample - loss: 40.0117 - mae: 40.0118 - mse: 18689.0762
Epoch 266/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 39.9180 - mae: 39.9180 - mse: 18751.4434
Epoch 267/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 40.1043 - mae: 40.1043 - mse: 18717.0820
Epoch 268/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 40.1455 - mae: 40.1455 - mse: 18778.2246
Epoch 269/1000


...

4000/4000 [==============================] - 2s 375us/sample - loss: 37.7675 - mae: 37.7675 - mse: 18644.2891
Epoch 717/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 37.6161 - mae: 37.6160 - mse: 18562.6836
Epoch 718/1000
4000/4000 [==============================] - 2s 434us/sample - loss: 37.8960 - mae: 37.8959 - mse: 18597.9121
Epoch 719/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 37.8160 - mae: 37.8160 - mse: 18741.0020
Epoch 720/1000
4000/4000 [==============================] - 1s 373us/sample - loss: 37.6659 - mae: 37.6659 - mse: 18619.5488
Epoch 721/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 37.6107 - mae: 37.6107 - mse: 18512.4863
Epoch 722/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 37.8665 - mae: 37.8665 - mse: 18633.7324
Epoch 723/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 37.5808 - mae: 37.5808 - mse: 18710.6348
Epoch 724/1000


4000/4000 [==============================] - 2s 383us/sample - loss: 37.6548 - mae: 37.6548 - mse: 18459.3320
Epoch 782/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 37.3305 - mae: 37.3305 - mse: 18471.9316
Epoch 783/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 37.3528 - mae: 37.3528 - mse: 18283.3145
Epoch 784/1000
4000/4000 [==============================] - 1s 375us/sample - loss: 37.1880 - mae: 37.1880 - mse: 18377.8184
Epoch 785/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 37.5754 - mae: 37.5754 - mse: 18413.9258
Epoch 786/1000
4000/4000 [==============================] - 2s 424us/sample - loss: 37.3823 - mae: 37.3823 - mse: 18341.6465
Epoch 787/1000
4000/4000 [==============================] - 2s 405us/sample - loss: 37.4766 - mae: 37.4766 - mse: 18348.7637
Epoch 788/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 37.5406 - mae: 37.5407 - mse: 18407.0488
Epoch 789/1000


4000/4000 [==============================] - 2s 387us/sample - loss: 36.7652 - mae: 36.7651 - mse: 18076.9941
Epoch 847/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 37.0013 - mae: 37.0013 - mse: 18124.1816
Epoch 848/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 37.2002 - mae: 37.2002 - mse: 18194.8496
Epoch 849/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 36.9575 - mae: 36.9575 - mse: 18099.1836
Epoch 850/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 36.9693 - mae: 36.9693 - mse: 18045.5215
Epoch 851/1000
4000/4000 [==============================] - 2s 435us/sample - loss: 36.9222 - mae: 36.9222 - mse: 18118.1523
Epoch 852/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 36.8908 - mae: 36.8908 - mse: 18114.9629
Epoch 853/1000
4000/4000 [==============================] - 2s 376us/sample - loss: 37.2090 - mae: 37.2090 - mse: 18113.1094
Epoch 854/1000


4000/4000 [==============================] - 1s 375us/sample - loss: 36.6599 - mae: 36.6599 - mse: 17940.6211
Epoch 912/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 36.5778 - mae: 36.5778 - mse: 17817.9824
Epoch 913/1000
4000/4000 [==============================] - 2s 403us/sample - loss: 36.8180 - mae: 36.8180 - mse: 18018.8496
Epoch 914/1000
4000/4000 [==============================] - 1s 375us/sample - loss: 36.5732 - mae: 36.5732 - mse: 17959.8184
Epoch 915/1000
4000/4000 [==============================] - 2s 426us/sample - loss: 36.4818 - mae: 36.4818 - mse: 17761.3301
Epoch 916/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 36.4418 - mae: 36.4418 - mse: 17829.7246
Epoch 917/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 36.7178 - mae: 36.7178 - mse: 17911.2285
Epoch 918/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 36.9733 - mae: 36.9733 - mse: 17860.7207
Epoch 919/1000


4000/4000 [==============================] - 2s 450us/sample - loss: 36.1749 - mae: 36.1749 - mse: 17773.2891
Epoch 977/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 36.2519 - mae: 36.2519 - mse: 17747.8223
Epoch 978/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 36.3424 - mae: 36.3424 - mse: 17740.3867
Epoch 979/1000
4000/4000 [==============================] - 2s 411us/sample - loss: 36.2521 - mae: 36.2521 - mse: 17699.7949
Epoch 980/1000
4000/4000 [==============================] - 2s 416us/sample - loss: 36.4870 - mae: 36.4870 - mse: 17632.4551
Epoch 981/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 36.2908 - mae: 36.2909 - mse: 17674.9648
Epoch 982/1000
4000/4000 [==============================] - 2s 381us/sample - loss: 36.4682 - mae: 36.4682 - mse: 17723.3652
Epoch 983/1000
4000/4000 [==============================] - 2s 441us/sample - loss: 36.3415 - mae: 36.3415 - mse: 17613.1523
Epoch 984/1000


In [9]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-35.3760761  -35.89249697 -33.22164787 -37.22204283 -32.76179496]
Mean MAE: -34.89481174516678
Standard Deviation of the mean: 1.6728558611214404
